# Importing modules and data

In [38]:
### Import libraries 
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D
import scipy 
import re
import os
import sys 
import matplotlib.dates
# import sqlite3
# import sqlalchemy

# import warnings
from datetime import datetime
from tabulate import tabulate
from getData import getData

acme, acme_raw = getData()


In [39]:
total_click=0
temp_click=np.zeros(len(acme.impression_id))
for i in range(1,len(acme.impression_id),1):
    total_click=total_click+acme.click[i]
    temp_click[i]=total_click
#    acme['Clickcount'][i]=total_click
print(temp_click)

[   0.    0.    0. ... 1373. 1373. 1373.]


In [41]:
acme['total_count']=temp_click
acme['total_count'][1000]

141.0

In [42]:
acme.corr()



,impression_id,click,number_of_vehicles,number_of_drivers,rank,policies_sold,married,insured,total_count
impression_id,1.000000,-0.017022,0.000910,0.001495,0.004129,-0.013871,-0.000908,0.012327,0.999781
click,-0.017022,1.000000,-0.260523,0.143265,-0.388226,0.601560,-0.073705,-0.041189,-0.016545
number_of_vehicles,0.000910,-0.260523,1.000000,-0.301658,0.666645,-0.179163,0.146629,0.000560,0.000910
number_of_drivers,0.001495,0.143265,-0.301658,1.000000,-0.414745,0.060568,-0.525229,-0.004529,0.001622
rank,0.004129,-0.388226,0.666645,-0.414745,1.000000,-0.250158,0.215103,0.083468,0.004124
policies_sold,-0.013871,0.601560,-0.179163,0.060568,-0.250158,1.000000,-0.048328,-0.078470,-0.013654
married,-0.000908,-0.073705,0.146629,-0.525229,0.215103,-0.048328,1.000000,0.006864,-0.001014
insured,0.012327,-0.041189,0.000560,-0.004529,0.083468,-0.078470,0.006864,1.000000,0.012459
total_count,0.999781,-0.016545,0.000910,0.001622,0.004124,-0.013654,-0.001014,0.012459,1.000000


In [40]:
acme.head()


,impression_id,click,number_of_vehicles,number_of_drivers,rank,policies_sold,married,insured
0,1,1,1,1,1,1,1,0
1,2,0,2,1,4,0,1,0
2,3,0,1,1,2,0,0,0
3,4,0,2,1,5,0,1,1
4,5,0,2,2,4,0,0,1


# Lasso

In [5]:
from sklearn.linear_model import Lasso

In [6]:
acme = acme[acme.click == 1]

y = acme['policies_sold']
X_temp = acme.copy().drop(['click', 'impression_id', 'policies_sold'], axis = 1)

X = X_temp.copy().drop(['number_of_vehicles', 'number_of_drivers'],axis = 1)

X['one_v'] = 2 - X_temp['number_of_vehicles']
X['two_v'] = X_temp['number_of_vehicles'] - 1

X['one_d'] = 2 - X_temp['number_of_drivers']
X['two_d'] = X_temp['number_of_drivers'] - 1

X['single'] = 1 - X_temp['married']
X['not_insured'] = 1 - X_temp['insured']

X

,rank,married,insured,total_count,one_v,two_v,one_d,two_d,single,not_insured
0,1,1,0,0.0,1,0,1,0,0,1
8,3,1,0,1.0,0,1,1,0,0,1
16,2,1,0,2.0,1,0,0,1,0,1
19,5,1,1,3.0,0,1,1,0,0,0
27,1,0,0,4.0,1,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...
9977,1,1,0,1369.0,1,0,0,1,0,1
9986,1,0,1,1370.0,1,0,0,1,1,0
9987,1,0,0,1371.0,1,0,0,1,1,1
9992,1,1,1,1372.0,1,0,0,1,0,0


In [29]:
#Zhan's original code for playing around
#alpha=0.00001,0.0001 seems more meaningful.
#alpha = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
#n = 9

# These will hold our coefficient estimates
#lasso_coefs = np.empty((len(alpha), n))

# for each alpha value
#for i in range(len(alpha)):    
    # set up the lasso pipeline
    
#    lasso = Lasso(alpha = alpha[i], normalize = True, max_iter = 1000000)
    # fit the lasso
#    lasso.fit(X, y)
    
    # record the coefficients
#    lasso_coefs[i,:] = lasso.coef_
alpha=0.0007
lasso = Lasso(alpha, normalize = True, max_iter = 1000000)
lasso.fit(X, y)
X['prediction'] = lasso.predict(X)



In [15]:
#pd.DataFrame(np.round(lasso_coefs,8),
#            index = ["alpha=" + str(a) for a in alpha])

In [30]:
for p in np.array(range(3100, 4700,10)) / 10000:
    z = X.prediction.apply(lambda x: 1 if x > p else 0)
    a = len(y[(y == 1) & (z == 1)])
    b = len(y[(y == 1) & (z == 0)])
    c = len(y[(y == 0) & (z == 1)])
    print("p value: %f" % p)
    if a + c != 0:
        pre = a / (a + c)
        print("    precision: %f" % pre)
        
    if a + b != 0:
        rec = a / (a + b)
        print("    recall: %f" % rec)
        
    if a + c != 0 and a + b != 0:
        print("    f1: %f" % (pre * rec / (pre + rec)))

p value: 0.310000
    precision: 0.451644
    recall: 0.856881
    f1: 0.295757
p value: 0.311000
    precision: 0.451644
    recall: 0.856881
    f1: 0.295757
p value: 0.312000
    precision: 0.451644
    recall: 0.856881
    f1: 0.295757
p value: 0.313000
    precision: 0.451644
    recall: 0.856881
    f1: 0.295757
p value: 0.314000
    precision: 0.451644
    recall: 0.856881
    f1: 0.295757
p value: 0.315000
    precision: 0.451644
    recall: 0.856881
    f1: 0.295757
p value: 0.316000
    precision: 0.451644
    recall: 0.856881
    f1: 0.295757
p value: 0.317000
    precision: 0.451644
    recall: 0.856881
    f1: 0.295757
p value: 0.318000
    precision: 0.451644
    recall: 0.856881
    f1: 0.295757
p value: 0.319000
    precision: 0.451644
    recall: 0.856881
    f1: 0.295757
p value: 0.320000
    precision: 0.451644
    recall: 0.856881
    f1: 0.295757
p value: 0.321000
    precision: 0.451644
    recall: 0.856881
    f1: 0.295757
p value: 0.322000
    precision: 0.47270

p value: 0.435000
    precision: 0.494083
    recall: 0.612844
    f1: 0.273546
p value: 0.436000
    precision: 0.494083
    recall: 0.612844
    f1: 0.273546
p value: 0.437000
    precision: 0.494083
    recall: 0.612844
    f1: 0.273546
p value: 0.438000
    precision: 0.494083
    recall: 0.612844
    f1: 0.273546
p value: 0.439000
    precision: 0.494083
    recall: 0.612844
    f1: 0.273546
p value: 0.440000
    precision: 0.494083
    recall: 0.612844
    f1: 0.273546
p value: 0.441000
    precision: 0.494083
    recall: 0.612844
    f1: 0.273546
p value: 0.442000
    precision: 0.494083
    recall: 0.612844
    f1: 0.273546
p value: 0.443000
    precision: 0.494083
    recall: 0.612844
    f1: 0.273546
p value: 0.444000
    precision: 0.494083
    recall: 0.612844
    f1: 0.273546
p value: 0.445000
    precision: 0.494083
    recall: 0.612844
    f1: 0.273546
p value: 0.446000
    precision: 0.494083
    recall: 0.612844
    f1: 0.273546
p value: 0.447000
    precision: 0.49408

In [10]:
print(X)

      rank  married  insured  total_count  one_v  two_v  one_d  two_d  single  \
0        1        1        0          0.0      1      0      1      0       0   
8        3        1        0          1.0      0      1      1      0       0   
16       2        1        0          2.0      1      0      0      1       0   
19       5        1        1          3.0      0      1      1      0       0   
27       1        0        0          4.0      1      0      0      1       1   
...    ...      ...      ...          ...    ...    ...    ...    ...     ...   
9977     1        1        0       1369.0      1      0      0      1       0   
9986     1        0        1       1370.0      1      0      0      1       1   
9987     1        0        0       1371.0      1      0      0      1       1   
9992     1        1        1       1372.0      1      0      0      1       0   
9996     1        1        0       1373.0      1      0      0      1       0   

      not_insured  predicti

# Ridge

In [33]:
from sklearn.linear_model import Ridge
ridge = Ridge()

In [36]:
ridge.fit(X=X[['rank', 'insured', 'not_insured', 'one_v', 'one_d'，'total_click']], y=y)

SyntaxError: invalid character in identifier (<ipython-input-36-459d93676155>, line 1)

In [37]:
X['prediction'] = ridge.predict(X[['rank', 'insured', 'not_insured', 'one_v', 'one_d']])

for p in np.array(range(4400, 4700,1)) / 10000:
    z = X.prediction.apply(lambda x: 1 if x > p else 0)
    a = len(y[(y == 1) & (z == 1)])
    b = len(y[(y == 1) & (z == 0)])
    c = len(y[(y == 0) & (z == 1)])
    print("p value: %f" % p)
    if a + c != 0:
        pre = a / (a + c)
        print("    precision: %f" % pre)
        
    if a + b != 0:
        rec = a / (a + b)
        print("    recall: %f" % rec)
        
    if a + c != 0 and a + b != 0:
        print("    f1: %f" % (pre * rec / (pre + rec)))

p value: 0.440000
    precision: 0.494083
    recall: 0.612844
    f1: 0.273546
p value: 0.440100
    precision: 0.494083
    recall: 0.612844
    f1: 0.273546
p value: 0.440200
    precision: 0.494083
    recall: 0.612844
    f1: 0.273546
p value: 0.440300
    precision: 0.494083
    recall: 0.612844
    f1: 0.273546
p value: 0.440400
    precision: 0.494083
    recall: 0.612844
    f1: 0.273546
p value: 0.440500
    precision: 0.494083
    recall: 0.612844
    f1: 0.273546
p value: 0.440600
    precision: 0.494083
    recall: 0.612844
    f1: 0.273546
p value: 0.440700
    precision: 0.494083
    recall: 0.612844
    f1: 0.273546
p value: 0.440800
    precision: 0.494083
    recall: 0.612844
    f1: 0.273546
p value: 0.440900
    precision: 0.494083
    recall: 0.612844
    f1: 0.273546
p value: 0.441000
    precision: 0.494083
    recall: 0.612844
    f1: 0.273546
p value: 0.441100
    precision: 0.494083
    recall: 0.612844
    f1: 0.273546
p value: 0.441200
    precision: 0.49408

p value: 0.452900
    precision: 0.501724
    recall: 0.533945
    f1: 0.258667
p value: 0.453000
    precision: 0.501724
    recall: 0.533945
    f1: 0.258667
p value: 0.453100
    precision: 0.501724
    recall: 0.533945
    f1: 0.258667
p value: 0.453200
    precision: 0.501724
    recall: 0.533945
    f1: 0.258667
p value: 0.453300
    precision: 0.501724
    recall: 0.533945
    f1: 0.258667
p value: 0.453400
    precision: 0.501724
    recall: 0.533945
    f1: 0.258667
p value: 0.453500
    precision: 0.501724
    recall: 0.533945
    f1: 0.258667
p value: 0.453600
    precision: 0.501724
    recall: 0.533945
    f1: 0.258667
p value: 0.453700
    precision: 0.501724
    recall: 0.533945
    f1: 0.258667
p value: 0.453800
    precision: 0.501724
    recall: 0.533945
    f1: 0.258667
p value: 0.453900
    precision: 0.501724
    recall: 0.533945
    f1: 0.258667
p value: 0.454000
    precision: 0.501724
    recall: 0.533945
    f1: 0.258667
p value: 0.454100
    precision: 0.50172

p value: 0.465400
    precision: 0.513619
    recall: 0.242202
    f1: 0.164589
p value: 0.465500
    precision: 0.513619
    recall: 0.242202
    f1: 0.164589
p value: 0.465600
    precision: 0.513619
    recall: 0.242202
    f1: 0.164589
p value: 0.465700
    precision: 0.513619
    recall: 0.242202
    f1: 0.164589
p value: 0.465800
    precision: 0.513619
    recall: 0.242202
    f1: 0.164589
p value: 0.465900
    precision: 0.513619
    recall: 0.242202
    f1: 0.164589
p value: 0.466000
    precision: 0.513619
    recall: 0.242202
    f1: 0.164589
p value: 0.466100
    precision: 0.513619
    recall: 0.242202
    f1: 0.164589
p value: 0.466200
    precision: 0.513619
    recall: 0.242202
    f1: 0.164589
p value: 0.466300
    precision: 0.513619
    recall: 0.242202
    f1: 0.164589
p value: 0.466400
    precision: 0.513619
    recall: 0.242202
    f1: 0.164589
p value: 0.466500
    precision: 0.513619
    recall: 0.242202
    f1: 0.164589
p value: 0.466600
    precision: 0.51361